In [415]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import scipy.io as spio
k = spio.loadmat('Data/matlab/emnist-mnist.mat') #change 'letters' to 'mnist', 'balanced', 'byclass', 'bymerge' or 'digits' 

In [416]:
training_data = np.matrix(k['dataset'][0][0][0][0][0][0])
lengt = len(training_data)
labels = k['dataset'][0][0][0][0][0][1]

size = len(np.unique(labels))
print(np.unique(labels))
labels = np.matrix(labels)
testing_data = np.matrix(k['dataset'][0][0][1][0][0][0])
testing_data_labels = np.matrix(k['dataset'][0][0][1][0][0][1])
training_data.shape

[0 1 2 3 4 5 6 7 8 9]


(60000, 784)

In [417]:
input_layer_size = training_data.shape[1]
output_layer_size = size
hidden_layer_size = 250
learning_rate = 0.05
e_init_lay1 = np.sqrt(6) / ((np.sqrt(input_layer_size)) + hidden_layer_size)
e_init_lay2 = np.sqrt(6)/ (np.sqrt(hidden_layer_size) + output_layer_size)
print(e_init_lay1)
print(e_init_lay2)

input_to_hidden_weight_matrix = np.matrix(2 * e_init_lay1 * np.random.rand(hidden_layer_size, input_layer_size)) - e_init_lay1
input_to_hidden_bias_vector = np.matrix(2 * e_init_lay1 * np.random.rand(hidden_layer_size)).T - e_init_lay1
hidden_to_output_weight_matrix = np.matrix(2 * e_init_lay2 * np.random.rand(output_layer_size, hidden_layer_size)) - e_init_lay2
hidden_to_output_bias_vector = np.matrix(2 * e_init_lay2 * np.random.rand(output_layer_size)).T - e_init_lay2

print(hidden_to_output_bias_vector)
print(hidden_to_output_weight_matrix)
print(input_to_hidden_weight_matrix)
print(input_to_hidden_bias_vector)




def change_labels_to_actual(labels, size):
    return_val = np.matlib.zeros((labels.shape[0], size))
    for i in range(labels.shape[0]):
        return_val[i, labels[i, 0]] = 1
    return return_val
labelp = change_labels_to_actual(labels, size)
testing_data_label = change_labels_to_actual(testing_data_labels, size)
labelp[0, :].shape
print((input_to_hidden_weight_matrix * training_data[0, :].T).shape)
    

0.00881111418267
0.0948995735616
[[-0.08279207]
 [ 0.06676678]
 [-0.02490754]
 [-0.05737897]
 [ 0.03488756]
 [-0.01758978]
 [-0.01586535]
 [-0.02228512]
 [-0.08589676]
 [-0.07806628]]
[[ 0.05004317  0.04064887 -0.0032867  ..., -0.0887481   0.02341749
   0.08924996]
 [ 0.09373914 -0.06829579  0.00543383 ...,  0.03137334  0.00900837
  -0.00128293]
 [-0.02738654  0.06182885 -0.08081667 ...,  0.02935118 -0.07888732
   0.09391152]
 ..., 
 [-0.00608862  0.06261471  0.06204737 ..., -0.07424761  0.00624031
   0.03317486]
 [ 0.06180467 -0.06012533  0.04403709 ..., -0.00315576  0.06620123
  -0.08300261]
 [ 0.0172702  -0.06337977  0.08554012 ...,  0.05260447  0.0698782
  -0.04385505]]
[[-0.00202598 -0.00409048  0.00802057 ...,  0.00331753 -0.00303903
   0.00084142]
 [-0.0054347  -0.00243663  0.00403735 ...,  0.0051274  -0.0040617
   0.00149533]
 [-0.00295054 -0.00179922  0.0062243  ..., -0.00208246 -0.0031559
   0.00525953]
 ..., 
 [ 0.00360665  0.0076611   0.0036361  ...,  0.00036395 -0.00132692

In [418]:
def sigmoid(npmat):
    
    return 1 / (1 + np.exp(-1 * npmat))

def apply_func_to_matrix(s, func):
    for i in range(s.shape[0]):
        for j in range(s.shape[1]):
            s[i, j] = func(s[i, j])
    return s

def relu(npmat):
    return 0.5 * abs(npmat) + .5* npmat

def flatten(one, two, three, four):
    o = np.array(np.ndarray.flatten(one))[0]
    t = np.array(np.ndarray.flatten(two))[0]
    th = np.array(np.ndarray.flatten(three))[0]
    fo = np.array(np.ndarray.flatten(four))[0]
    ot = np.append(o, t)
    otth = np.append(ot, th)
    otthfo = np.append(otth, fo)
    return otthfo

def unflatten(thet):
    intohid = np.matrix(thet[:input_layer_size * hidden_layer_size]).reshape(hidden_layer_size, input_layer_size)
    thet = thet[input_layer_size * hidden_layer_size:]
    inbias = np.matrix(thet[:hidden_layer_size])
    thet = thet[hidden_layer_size:]
    hittoout = np.matrix(thet[:hidden_layer_size * output_layer_size].reshape(output_layer_size, hidden_layer_size))
    thet = np.matrix(thet[hidden_layer_size * output_layer_size:])
    return intohid, inbias.T, hittoout, thet.T

print(np.log(np.matrix([[1, 4], [2, 5]])))

[[ 0.          1.38629436]
 [ 0.69314718  1.60943791]]


In [407]:
def compute_predict_layer1(input_layer):
    lay1 = input_to_hidden_weight_matrix * input_layer + input_to_hidden_bias_vector * np.matrix(np.ones(input_layer.shape[1]))
    return lay1
def compute_activation_layer(activation, layer):
    return activation(layer)
def compute_predict_layer2(lay1):
    lay2 = hidden_to_output_weight_matrix * lay1 + hidden_to_output_bias_vector * np.matrix(np.ones(lay1.shape[1]))
    return lay2

def compute_cost_cross_entropy(true_values, predicted_values):
    sum = 0
    
    for k in range(true_values.shape[0]):
        
       
        sum+= np.log(predicted_values[k, 0]) * true_values[k, 0] + (1 - true_values[k, 0]) * np.log(1 - predicted_values[k, 0])
        
    
    return -1 * sum
def compute_cost_mean_squared_error(true_values, predicted_values):
    return np.matrix(np.apply_along_axis(sum_of_squares, 1, np.array(true_values - predicted_values)))
    
def sum_of_squares(arr):
    return sum([x*x for x in arr])

def gradient_mean_squared_error(true_values, predicted_values):
    return 2* (true_values - predicted_values)



def gradient_cross_entropy(true_values, activation_three):
    return  activation_three - true_values

def gradient_bias_layer1(delta_3, activation_gradient_of_z):
    
    return np.multiply(hidden_to_output_weight_matrix.T * delta_3, activation_gradient_of_z)

def relu_gradient(layer):
    def relu_grad_one(num):
        if num > 0:
            return 1.0
        else:
            return 0.0
    return apply_func_to_matrix(layer, relu_grad_one)

def sigmoid_gradient(layer):
    def sigmoid_grad_one(num):
        return (1 - sigmoid(num)) * sigmoid(num)
    return apply_func_to_matrix(layer, sigmoid_grad_one)
def compute_gradient_bias_layer2(predicted_values, activation_three, grad_function):
    return grad_function(predicted_values, activation_three)
def compute_gradient_hidden_to_output(delta_3, a_2):
    return delta_3 * a_2.T
def compute_gradient_input_to_hidden(delta_2, a_1):
    return delta_2 * a_1.T

def comp_grad(inpt, true_values, activation1, activation2, eval_f, eval_f_grad, need_extra_grad, activation_gradient1, activation_gradient2):
    lay1 = compute_predict_layer1(inpt)
    
    act1 = compute_activation_layer(activation1, lay1)
    lay2 = compute_predict_layer2(act1)
    act2 = compute_activation_layer(activation2, lay2)
    
    #loss = eval_f(act2, true_values)
    
    delta_3 = eval_f_grad(true_values, act2)
    
    if need_extra_grad:
        delta_3 = activation_gradient2(delta_3)
    grad_hidden_output = compute_gradient_hidden_to_output(delta_3, act1)
    g_prime_of_z = activation_gradient1(lay1)
    delta_2 = gradient_bias_layer1(delta_3, g_prime_of_z)
    grad_input_hidden = compute_gradient_input_to_hidden(delta_2, inpt)
    
   
    
    return flatten(grad_input_hidden, delta_2, grad_hidden_output, delta_3)
    
def comp_loss(inpt, true_values, activation1, activation2, eval_f):
    lay1 = compute_predict_layer1(inpt)
    
    act1 = compute_activation_layer(activation1, lay1)
    lay2 = compute_predict_layer2(act1)
    act2 = compute_activation_layer(activation2, lay2)
    
    loss = eval_f(act2, true_values)
    return loss

def grad_check(inpt, true_values, activation1, activation2, eval_f, epsilon):
        lay1 = compute_predict_layer1(inpt)
        act1 = compute_activation_layer(activation1, lay1)
        lay2 = compute_predict_layer2(act1)
        act2 = compute_activation_layer(activation2, lay2)
        loss_reg = eval_f(true_values, act2)
        
        def calc_grad(mat):
            return_mat = np.matlib.zeros(mat.shape)
            for i in range(mat.shape[0]):
                
                for j in range(mat.shape[1]):
                    mat[i, j] += epsilon
                    lay1 = compute_predict_layer1(inpt)
                    act1 = compute_activation_layer(activation1, lay1)
                    lay2 = compute_predict_layer2(act1)
                    act2 = compute_activation_layer(activation2, lay2)
                    return_mat[i, j] = (eval_f(true_values, act2) - loss_reg) / epsilon
                    mat[i, j] -= epsilon
            return return_mat
        #return calc_grad(input_to_hidden_weight_matrix), calc_grad(input_to_hidden_bias_vector),calc_grad(hidden_to_output_weight_matrix), calc_grad(hidden_to_output_bias_vector)
        return calc_grad(input_to_hidden_weight_matrix)
#print(comp_grad(training_data[0, :].T, labelp[0, :].T, sigmoid, sigmoid, compute_cost_cross_entropy, gradient_cross_entropy, False, sigmoid_gradient, sigmoid_gradient))


#print(grad_check(training_data[0, :].T, labelp[0, :].T, sigmoid, sigmoid, compute_cost_cross_entropy, 0.0001))

In [408]:
def compute_total_gradient(thet, lamb, training_data, true_values, activation1, activation2, eval_f, eval_f_grad, need_extra_grad, activation_gradient1, activation_gradient2):
    summ = np.zeros(len(thet))
    for i in range(training_data.shape[0]):
        summ += comp_grad(training_data[i, :].T, true_value[0, :].T, sigmoid, sigmoid, compute_cost_cross_entropy, gradient_cross_entropy, False, sigmoid_gradient, sigmoid_gradient)
    summ += thet * lamb
    avg = summ / training_data.shape[0]

def comp_total_loss(thet, lamb, training_data, true_values, activation1, activation2, eval_f):
    summ = np.zeros(len(thet))
    for i in range(training_data.shape[0]):
        summ += comp_loss(training_data[i, :].T, true_values[i, :].T, activation1, activation2, eval_f)
    summ += sum([s*s for s in thet]) * lamb
    avg = summ / training_data.shape[0]
    

In [409]:
init_theta = flatten(input_to_hidden_weight_matrix, input_to_hidden_bias_vector, hidden_to_output_weight_matrix, hidden_to_output_bias_vector)

lamb = 0.01

def cost_func_and_gradient(theta, *args):
    lamb = args[0]
    training_data = args[1]
    labelp = args[2]
    input_to_hidden_weight_matrix, input_to_hidden_bias_vector, hidden_to_output_weight_matrix, hidden_to_output_bias_vector = unflatten(theta)
    thet = flatten(input_to_hidden_weight_matrix, np.matlib.zeros(input_to_hidden_bias_vector.shape), hidden_to_output_weight_matrix, np.matlib.zeros(hidden_to_output_bias_vector.shape))
    
    def compute_total_loss_gradient(thet, lamb, training_data, true_values, activation1, activation2, eval_f, eval_f_grad, need_extra_grad, activation_gradient1, activation_gradient2):
        
        summ = np.zeros(len(thet))
        sump = 0.0
        for i in range(training_data.shape[0]):
            k = comp_grad(training_data[i, :].T, true_values[i, :].T, activation1, activation2, eval_f, eval_f_grad, need_extra_grad, activation_gradient1, activation_gradient2)
            summ += k[1]
            sump += k[0]
            
        
        summ += thet* lamb
        
        sump += sum([s *s for s in thet]) * lamb
        avgp = sump / training_data.shape[0]
        avgm = summ / training_data.shape[0]
        
        return avgp, avgm
        
    def compute_predict_layer1(input_layer):
        lay1 = input_to_hidden_weight_matrix * input_layer + input_to_hidden_bias_vector * np.matrix(np.ones(input_layer.shape[1]))
        return lay1
    def compute_activation_layer(activation, layer):
        return activation(layer)
    def compute_predict_layer2(lay1):
        lay2 = hidden_to_output_weight_matrix * lay1 + hidden_to_output_bias_vector * np.matrix(np.ones(lay1.shape[1]))
        return lay2

    def compute_cost_cross_entropy(true_values, predicted_values):
        sum = 0

        for k in range(true_values.shape[0]):


            sum+= np.log(predicted_values[k, 0]) * true_values[k, 0] + (1 - true_values[k, 0]) * np.log(1 - predicted_values[k, 0])


        return -1 * sum
    def sigmoid(npmat):
        
        
        return 1 / (1 + np.exp(-1 * np.matrix(np.clip(npmat, -500, 500 ))))

    def apply_func_to_matrix(s, func):
        for i in range(s.shape[0]):
            for j in range(s.shape[1]):
                s[i, j] = func(s[i, j])
        return s

    def relu(npmat):
        return 0.5 * abs(npmat) + .5* npmat
    def compute_cost_mean_squared_error(true_values, predicted_values):
        return np.matrix(np.apply_along_axis(sum_of_squares, 1, np.array(true_values - predicted_values)))

    def sum_of_squares(arr):
        return sum([x*x for x in arr])

    def gradient_mean_squared_error(true_values, predicted_values):
        return 2* (true_values - predicted_values)



    def gradient_cross_entropy(true_values, activation_three):
        return  activation_three - true_values

    def gradient_bias_layer1(delta_3, activation_gradient_of_z):

        return np.multiply(hidden_to_output_weight_matrix.T * delta_3, activation_gradient_of_z)

    def relu_gradient(layer):
        def relu_grad_one(num):
            if num > 0:
                return 1.0
            else:
                return 0.0
        return apply_func_to_matrix(layer, relu_grad_one)

    def sigmoid_gradient(layer):
        def sigmoid_grad_one(num):
            return (1 - sigmoid(num)) * sigmoid(num)
        return apply_func_to_matrix(layer, sigmoid_grad_one)
    def compute_gradient_bias_layer2(predicted_values, activation_three, grad_function):
        return grad_function(predicted_values, activation_three)
    def compute_gradient_hidden_to_output(delta_3, a_2):
        return delta_3 * a_2.T
    def compute_gradient_input_to_hidden(delta_2, a_1):
        return delta_2 * a_1.T

    def comp_grad(inpt, true_values, activation1, activation2, eval_f, eval_f_grad, need_extra_grad, activation_gradient1, activation_gradient2):
        lay1 = compute_predict_layer1(inpt)

        act1 = compute_activation_layer(activation1, lay1)
        lay2 = compute_predict_layer2(act1)
        act2 = compute_activation_layer(activation2, lay2)
        

        loss = eval_f(true_values, act2)
        
        delta_3 = eval_f_grad(true_values, act2)

        if need_extra_grad:
            delta_3 = activation_gradient2(delta_3)
        grad_hidden_output = compute_gradient_hidden_to_output(delta_3, act1)
        g_prime_of_z = activation_gradient1(lay1)
        delta_2 = gradient_bias_layer1(delta_3, g_prime_of_z)
        grad_input_hidden = compute_gradient_input_to_hidden(delta_2, inpt)
        
    

        return loss, flatten(grad_input_hidden, delta_2, grad_hidden_output, delta_3)
    
    

    return compute_total_loss_gradient(thet, lamb, training_data, labelp, sigmoid, sigmoid, compute_cost_cross_entropy, gradient_cross_entropy, False, sigmoid_gradient, sigmoid_gradient)

In [410]:
def cost_func(theta, *args):
    return cost_func_and_gradient(theta, *args)[0]
def gradient(theta, *args):
    return cost_func_and_gradient(theta, *args)[1]

In [411]:
alpha = 0.00005
A = np.hstack((training_data, labels))
theta = init_theta
for _ in range(10000):
    print(_)
    batch_xs_ys = A[np.random.choice(A.shape[0], 100), :]
    batch_xs = batch_xs_ys[:, :-1]
    batch_ys = batch_xs_ys[:, -1]
    z = np.matlib.zeros((len(batch_ys), size))
    for i in range(len(batch_ys)):
        z[i, batch_ys[i, 0]] = 1
    val = cost_func_and_gradient(theta, lamb, batch_xs, z)
    print(val[0])
    theta = theta - alpha * val[1]
    

   

0


/Users/aagarwal_601/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log


inf
1
13.0100404544
2
9.72599054032
3
8.1051171028
4
6.09244689758
5
5.16716885243
6
4.72728309357
7
3.98595323359
8
3.26254856333
9
2.77586865554
10
2.6847273137
11
3.49502626109
12
2.59308444383
13
2.51970523237
14
2.496393956
15
2.47282076107
16
2.41334906686
17
2.77673283859
18
2.34766048284
19
2.02364569783
20
1.63688224749
21
1.48088459104
22
2.58443939133
23
2.03209183893
24
1.8494824817
25
1.96371491555
26
1.94584218339
27
1.96931667726
28
1.7050102181
29
2.2104815332
30
1.99041615938
31
2.06345046452
32
1.39896061202
33
1.56655038954
34
1.61731517375
35
1.71765460843
36
1.43957398799
37
1.73687207334
38
1.68343030909
39
1.96153997689
40
1.5232050714
41
1.78035548276
42
1.53472564248
43
1.57339654924
44
1.32347402132
45
1.36870698281
46
1.57521155563
47
1.20687523594
48
1.51505877583
49
1.20410385693
50
1.19269567556
51
1.33435173083
52
1.50727035591
53
1.14518255799
54
1.65456782516
55
1.2412026455
56
0.997504473693
57
1.51778227929
58
0.967092919815
59
1.53989272859
60
1.4655

1.21248846333
449
0.428763949399
450
0.637209140354
451
0.646399377029
452
0.705149765179
453
0.650690546299
454
0.640911544227
455
0.469870401647
456
0.660585512939
457
0.374509577013
458
0.594998169659
459
0.439443261789
460
0.726710283317
461
0.545145031487
462
0.865210060736
463
0.531107083723
464
0.527909758579
465
0.565040329506
466
0.526290300147
467
0.642908609982
468
0.532452518361
469
0.616378778436
470
0.515900844466
471
0.419604087747
472
0.504476541378
473
0.362379334643
474
0.520418809903
475
0.555976521342
476
0.534777590121
477
0.340671367191
478
0.415393094223
479
0.485692969248
480
0.47817908224
481
0.466213984426
482
0.414291719637
483
0.574100826576
484
0.450934002797
485
0.616424885277
486
0.628538544697
487
0.245602487306
488
0.654924698307
489
0.783291635906
490
0.445318676389
491
0.564970418868
492
0.591225523001
493
0.620193508318
494
0.260232983958
495
0.254087955106
496
0.720719337732
497
0.57302155288
498
0.515645445536
499
0.723811838188
500
0.34156804876
5

0.656784638698
884
0.593392806991
885
0.518875487114
886
0.602738102101
887
0.310785630445
888
0.294734015635
889
0.581090318924
890
0.600885701455
891
0.489522742429
892
0.669314650209
893
0.348454728547
894
0.334767629161
895
0.264424400321
896
0.264899004711
897
0.444539944238
898
0.2376747738
899
0.37797453992
900
0.228190944634
901
0.253752135767
902
0.473434001958
903
0.273818296243
904
0.515568945304
905
0.212931403892
906
0.343544233472
907
0.362270237396
908
0.478052148423
909
0.322121368224
910
0.23208410145
911
0.31611554105
912
0.472930833783
913
0.459110326966
914
0.450747992016
915
0.527775564975
916
0.268488126133
917
0.476736389423
918
0.621581224235
919
0.240345937774
920
0.223786706179
921
0.379887183618
922
0.309126492555
923
0.345195672622
924
0.316496668467
925
0.422899235776
926
0.298337882394
927
0.385742209154
928
0.373878885616
929
0.373590973103
930
0.319918229519
931
0.237352528991
932
0.355871963387
933
0.560382546766
934
0.580872565689
935
0.53214522406
936

KeyboardInterrupt: 

In [ ]:
print(theta)


In [412]:
print(theta)
alpha = 0.000005
A = np.hstack((training_data, labels))
for _ in range(10000):
    print(_)
    batch_xs_ys = A[np.random.choice(A.shape[0], 100), :]
    batch_xs = batch_xs_ys[:, :-1]
    batch_ys = batch_xs_ys[:, -1]
    z = np.matlib.zeros((len(batch_ys), size))
    for i in range(len(batch_ys)):
        z[i, batch_ys[i, 0]] = 1
    val = cost_func_and_gradient(theta, lamb, batch_xs, z)
    print(val[0])
    theta = theta - alpha * val[1]

[-0.00368629  0.00761133 -0.00047078 ..., -0.06005936 -0.07524761
  0.05390293]
0
0.465940606015
1
0.387664089795
2
0.441308110128
3
0.439668818181
4
0.486661022894
5
0.491912921692
6
0.323566495962
7
0.41881225394
8
0.331358491856
9
0.397990630301
10
0.312310050858
11
0.315671343361
12
0.369147066735
13
0.353938629031
14
0.304863731217
15
0.396332582117
16
0.47118573097
17
0.458247674932
18
0.386886847819
19
0.55067950471
20
0.506839825492
21
0.220919994159
22
0.363225114109
23
0.226437375684
24
0.293232829788
25
0.279269851365
26
0.4495251184
27
0.314260806567
28
0.234427602216
29
0.376297956125
30
0.941673930211
31
0.347529090828
32
0.534894850224
33
0.45862134441
34
0.283162940252
35
0.201043529336
36
0.327732720899
37
0.350479033202
38
0.494399647822
39
0.412465146066
40
0.58664813928
41
0.231195455563
42
0.334652322454
43
0.356190230416
44
0.521661612982
45
0.369718944387
46
0.452884465578
47
0.361523508878
48
0.433770229873
49
0.249694152154
50
0.606542517138
51
0.284240726107
5

KeyboardInterrupt: 

In [414]:
important = theta
print(theta)

def test(theta, data, labels):
    input_to_hidden_weight_matrix, input_to_hidden_bias_vector, hidden_to_output_weight_matrix, hidden_to_output_bias_vector = unflatten(theta)
    
    def compute_predict_layer1(input_layer):
        lay1 = input_to_hidden_weight_matrix * input_layer + input_to_hidden_bias_vector * np.matrix(np.ones(input_layer.shape[1]))
        return lay1
    def compute_activation_layer(activation, layer):
        return activation(layer)
    def compute_predict_layer2(lay1):
        lay2 = hidden_to_output_weight_matrix * lay1 + hidden_to_output_bias_vector * np.matrix(np.ones(lay1.shape[1]))
        return lay2
    def sigmoid(npmat):
        
        
        return 1 / (1 + np.exp(-1 * np.matrix(np.clip(npmat, -500, 500 ))))
    accuracy = 0
    total = data.shape[0]
    for i in range(data.shape[0]):
        lay1 = compute_predict_layer1(data[i, :].T)
        act1 = compute_activation_layer(sigmoid, lay1)
        lay2 = compute_predict_layer2(act1)
        final = np.array(compute_activation_layer(sigmoid, lay2).T)[0]
       
        if np.argmax(final) == labels[i, 0]:
        
            accuracy +=1
        
    print(accuracy / total)
    return accuracy / total     

test(theta, testing_data, testing_data_labels)  

[-0.00368629  0.00761133 -0.00047078 ..., -0.06005942 -0.07524809
  0.05390216]
0.4722


0.4722

In [367]:
important = theta
print(theta)

def test(theta, data, labels):
    input_to_hidden_weight_matrix, input_to_hidden_bias_vector, hidden_to_output_weight_matrix, hidden_to_output_bias_vector = unflatten(theta)
    
    def compute_predict_layer1(input_layer):
        lay1 = input_to_hidden_weight_matrix * input_layer + input_to_hidden_bias_vector * np.matrix(np.ones(input_layer.shape[1]))
        return lay1
    def compute_activation_layer(activation, layer):
        return activation(layer)
    def compute_predict_layer2(lay1):
        lay2 = hidden_to_output_weight_matrix * lay1 + hidden_to_output_bias_vector * np.matrix(np.ones(lay1.shape[1]))
        return lay2
    def sigmoid(npmat):
        
        
        return 1 / (1 + np.exp(-1 * np.matrix(np.clip(npmat, -500, 500 ))))
    accuracy = 0
    total = data.shape[0]
    for i in range(data.shape[0]):
        lay1 = compute_predict_layer1(data[i, :].T)
        act1 = compute_activation_layer(sigmoid, lay1)
        lay2 = compute_predict_layer2(act1)
        final = np.array(compute_activation_layer(sigmoid, lay2).T)[0]
       
        if np.argmax(final) == labels[i, 0]:
        
            accuracy +=1
        
    print(accuracy / total)
    return accuracy / total     

test(theta, testing_data, testing_data_labels)  

[ 0.00461464  0.00746492 -0.0069617  ..., -0.03016109 -0.10424411
 -0.12719614]
0.9295


0.9295

In [ ]:
#THIS IS THE ACCURACY FOR THE MNIST DATABASE